# Read data and housekeeping

In [171]:
import pandas as pd
import os
saveDir = '/Users/AAA/whereYouWishToSaveOutput'
wbName = 'Attendance 2020-05-18.xlsx' # file to read

attendance = pd.read_excel(os.path.join(saveDir, wbName))

# Data formatting

In [173]:
attendance['status'] = attendance.status.map({True: 'Present', False: 'Absent'})

# Parliament attendance rate bin range by year

In [16]:
attRate = attendance.groupby([attendance.sittingDt.dt.year,'members','status'])['members'].count().unstack().reset_index()
attRate = attRate.fillna(0)
attRate['Rate'] = attRate.Present / (attRate.Present + attRate.Absent)

In [85]:
import numpy as np

# setting histogram
hist = pd.DataFrame()
for y in attRate.sittingDt.unique():
    hist_val, bin_edges = np.histogram(attRate.loc[attRate.sittingDt == y,'Rate'], 
                                        bins = (0,0.3,0.6,0.9,1,1.1))
    hist[y] = hist_val

# set index
hist = hist.set_index([[">= 0 and < 0.3", ">= 0.3 and <0.6", ">=0.6 and < 0.9",
               ">= 0.90 and <1", ">= 1"]])

# calculate by percentage
hist_per = hist.apply(lambda x: x/x.sum())



In [86]:
display(hist)
display(hist_per)

,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
>= 0 and < 0.3,0,0,0,0,1,0,0,0,0,1,1,1,0,0,0,2
>= 0.3 and <0.6,3,4,1,3,3,2,2,2,1,2,0,1,2,1,2,0
>=0.6 and < 0.9,44,26,29,35,36,35,28,41,27,45,18,25,31,21,30,24
>= 0.90 and <1,36,26,44,31,33,29,14,24,33,36,34,36,27,38,33,9
>= 1,11,62,19,23,26,25,76,33,38,24,46,37,37,49,35,65


,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
>= 0 and < 0.3,0.000000,0.000000,0.000000,0.000000,0.010101,0.000000,0.000000,0.00,0.000000,0.009259,0.010101,0.01,0.000000,0.000000,0.00,0.02
>= 0.3 and <0.6,0.031915,0.033898,0.010753,0.032609,0.030303,0.021978,0.016667,0.02,0.010101,0.018519,0.000000,0.01,0.020619,0.009174,0.02,0.00
>=0.6 and < 0.9,0.468085,0.220339,0.311828,0.380435,0.363636,0.384615,0.233333,0.41,0.272727,0.416667,0.181818,0.25,0.319588,0.192661,0.30,0.24
>= 0.90 and <1,0.382979,0.220339,0.473118,0.336957,0.333333,0.318681,0.116667,0.24,0.333333,0.333333,0.343434,0.36,0.278351,0.348624,0.33,0.09
>= 1,0.117021,0.525424,0.204301,0.250000,0.262626,0.274725,0.633333,0.33,0.383838,0.222222,0.464646,0.37,0.381443,0.449541,0.35,0.65


In [87]:
fileName = 'Histogram over the years.xlsx'
hist.T.reset_index().to_excel(os.path.join(saveDir, fileName), index = False)

In [83]:
fileName = 'Histogram over the years.xlsx'
hist_per.T.reset_index().to_excel(os.path.join(saveDir, fileName), index = False)

# Average attendance

In [93]:
avgAtt = attRate.groupby('sittingDt')['Rate'].agg(['min','mean','median','max']).reset_index()

In [95]:
fileName = 'Attendance statistics.xlsx'
avgAtt.to_excel(os.path.join(saveDir, fileName), index = False)

# Who attended most session

In [179]:
# get first parliament
attendance['parliamentNum'] = attendance.parliament.str.extract('([0-9]*)th')
attendance['parliamentNum'] = attendance.parliamentNum.astype('int64')
firstPar = attendance.groupby('members')['parliamentNum'].agg(['min','max']).reset_index()
firstPar['parliamentCount'] = firstPar['max'] - firstPar['min'] + 1

In [159]:
# calculate individual attendance rate 
indRate = attRate.groupby(['members'])['Absent','Present'].agg('sum').reset_index()
indRate['Total'] = indRate.Present + indRate.Absent
indRate['Rate'] = indRate.Present / indRate.Total
indRate = indRate.merge(firstPar, how = 'left')
indRate = indRate.sort_values(['Total','Rate','min'], ascending = False)

In [161]:
indRate.groupby(['min']).head().sort_values(['min','Rate','Total'], ascending = False)

,members,Absent,Present,Total,Rate,min,max,parliamentCount
46,Dennis Tan Lip Fong,0.0,130.0,130.0,1.000000,13,13,1
47,Desmond Choo,0.0,130.0,130.0,1.000000,13,13,1
123,Louis Ng Kok Kwang,0.0,130.0,130.0,1.000000,13,13,1
133,Melvin Yong Yik Chye,1.0,129.0,130.0,0.992308,13,13,1
211,Yee Chia Hsing,1.0,129.0,130.0,0.992308,13,13,1
101,Lawrence Wong,2.0,245.0,247.0,0.991903,12,13,2
159,Pritam Singh,4.0,243.0,247.0,0.983806,12,13,2
11,Ang Wei Neng,5.0,242.0,247.0,0.979757,12,13,2
80,Janil Puthucheary,7.0,240.0,247.0,0.971660,12,13,2
155,Patrick Tay Teck Guan,7.0,240.0,247.0,0.971660,12,13,2


In [167]:
indRate['Total_grp'] = indRate.Total // 100 * 100

In [169]:
indRate.groupby(['Total_grp']).head().sort_values(['Total_grp','Rate','Total'], ascending = False)

,members,Absent,Present,Total,Rate,min,max,parliamentCount,Total_grp
104,Lee Hsien Loong,5.0,415.0,420.0,0.988095,10,13,4,400.0
125,Low Thia Khiang,8.0,412.0,420.0,0.980952,10,13,4,400.0
57,Gan Kim Yong,13.0,407.0,420.0,0.969048,10,13,4,400.0
68,Heng Chee How,14.0,406.0,420.0,0.966667,10,13,4,400.0
91,Khaw Boon Wan,23.0,397.0,420.0,0.945238,10,13,4,400.0
45,Denise Phua Lay Peng,4.0,370.0,374.0,0.989305,11,13,3,300.0
38,Christopher De Souza,6.0,368.0,374.0,0.983957,11,13,3,300.0
113,Lim Biow Chuan,7.0,367.0,374.0,0.981283,11,13,3,300.0
111,Liang Eng Hwa,9.0,365.0,374.0,0.975936,11,13,3,300.0
17,Baey Yam Keng,15.0,359.0,374.0,0.959893,11,13,3,300.0


## Number of attendance over the years

In [428]:
images = pd.read_excel(f'{saveDir}/page source/members_images.xlsx')
images = images.sort_values('page')
images = images.drop_duplicates(subset = ['names','parties','imageLink','parliament'], keep = 'last')

# format names
images['names'] = images['names'].str.replace('\(.*\)',"")
images['names'] = images.names.str.replace('\.|,|Prof|Dr|Assoc',"")
images['names'] = images.names.str.replace('-'," ")
images['names'] = images.names.apply(lambda x: x.strip())

# format link
images['imageLink'] = 'https://www.parliament.gov.sg' + images.imageLink

# map names
images.head(3)

,names,parties,imageLink,parliament,page
75,Png Paak Liang Ivan,Nominated Member of Parliament,https://www.parliament.gov.sg/images/default-s...,10th,10th
73,Othman Bin Haron Eusofe,People's Action Party,https://www.parliament.gov.sg/images/default-s...,"5th, 6th, 7th, 8th, 9th, 10th",10th
72,Ong Soh Khim,Nominated Member of Parliament,https://www.parliament.gov.sg/images/default-s...,10th,10th


In [429]:
# some names are entered differently. map them
mapping = {
    'Abdullah Bin Tarmugi' : 'Abdullah Tarmugi',
    'Mohamed Maidin BPM': 'Mohamad Maidin B P M',
    'Lim Chun Leng Michael': 'Michael Lim Chun Leng',
    'Leow Chin Hin Lawrence':'Lawrence Leow Chin Hin',
    'Chen Seow Phun John': 'John Chen Seow Phun',
    'Png Paak Liang Ivan': 'Ivan Png Paak Liang',
    'Lee Warren': 'Warren Lee',
    'Tan Keng Yam Tony': 'Tony Tan Keng Yam',
    'Chia Kiah Hong Steve': 'Steve Chia Kiah Hong',
    'Lim T E David': 'David Lim T E',
    'Gan Lai Chiang Andy': 'Andy Gan Lai Chiang',
    'Ahmad Khalis bin Abdul Gahni': 'Ahmad Khalis Bin Abdul Ghani',
    'Yeo Yong Boon George': 'George Yong Boon Yeo',
    'Yu Foo Yee Shoon': 'Yu Foo Yee Shoon',
    'Chan Meng Wah Alexander': 'Alexander Chan Meng Wah',
    'Christopher de Souza': 'Christopher De Souza',
    'Masagos Zulkifli Bin Masagos Mohamad': 'Masagos Zulkifli Bmm',
    'Thio Li ann': 'Thio Li Ann',
    'Khew Teck Fook Edwin': 'Edwin Khew Teck Fook',
    'Hri Kumar Nair': 'Hri Kumar',
    'Kalyani K Mehta': 'Kalyani K  Mehta',
    'Desmond Lee': 'Desmond Lee Ti Seng',
    'Muhamad Faisal Bin Abdul Manap': 'Muhamad Faisal Abdul Manap'
}

images['namesMapped'] = images['names'].map(mapping).fillna(images['names'])

In [430]:
attendance['minPar'] = attendance.groupby('members')['parliamentNum'].transform('min')

In [446]:
# count number of attendance each year
presentCount = attendance[attendance.status == 'Present'].groupby(['members','minPar',attendance.sittingDt.dt.year])['status'].count()
presentCount = presentCount.reset_index().sort_values(['members','sittingDt','status'])

# standard years
minYr, maxYr = presentCount.sittingDt.agg(['min','max'])
presentCount = presentCount.set_index('sittingDt')
presentCount = presentCount[['members','status']].groupby('members')['status'].apply(lambda x: x.reindex(range(minYr, maxYr+1)))
presentCount = presentCount.reset_index()
presentCount = presentCount.fillna(0)

# get agg
presentCount['agg'] = presentCount.groupby('members')['status'].transform(lambda x: x.cumsum())


# tranpose it for visualization
presentCount = presentCount.pivot_table(index = ['members'],columns = 'sittingDt', values = 'agg').reset_index()
presentCount = presentCount.fillna(0)

# merge with profiles
presentCount = presentCount.merge(images[['namesMapped','imageLink','parties']], how = 'left', 
                                  left_on = 'members',right_on = 'namesMapped')

In [447]:
fileName = 'Present count.xlsx'
presentCount.to_excel(os.path.join(saveDir, fileName), index = False)

# Total proceeding and attendance Rate

In [512]:
images['firstParliament'] = images.parliament.apply(lambda x: x.split(',')[0])
images['firstParMap'] = images.firstParliament.copy()
images.loc[images.firstParMap.str.len() <= 3, 'firstParMap'] = "Before 10th"

In [516]:
indRate = indRate.merge(images[['namesMapped','firstParMap','parties']], how = 'left',
                       left_on = 'members', right_on = 'namesMapped')

In [517]:
fileName = 'Individual Rate.xlsx'
indRate[['members','Present','Total','Rate','firstParMap','parties']].to_excel(os.path.join(saveDir, fileName), index = False)

In [515]:
indRate = indRate.drop(['namesMapped','firstParMap','parties'], axis = 1)

In [556]:
indRate.groupby('firstParMap')['Rate'].mean()

firstParMap
10th           0.897941
11th           0.905139
12th           0.928007
13th           0.921702
Before 10th    0.872278
Name: Rate, dtype: float64

# Attendance by geography

In [539]:
# count attendance rate by parliament 
attRate2 = attendance.groupby(['parliamentNum','members','area','status'])['members'].count().unstack().reset_index()
attRate2 = attRate2.fillna(0)
attRate2['Rate'] = attRate2.Present / (attRate2.Present + attRate2.Absent)

# get latest parliament
print(f"attRate is attendance for parliament {attRate2.parliamentNum.max()}")
attRate2 = attRate2.loc[attRate2.parliamentNum == attRate2.parliamentNum.max()]

attRate is attendance for parliament 13


In [551]:
# calculate individual attendance rate 
indRate2 = attRate2.groupby(['members','area'])['Absent','Present'].agg('sum').reset_index()
indRate2['Total'] = indRate2.Present + indRate2.Absent
indRate2['Rate'] = indRate2.Present / indRate2.Total
indRate2 = indRate2.sort_values(['Total','Rate'], ascending = False)

# get
areaRate = indRate2.groupby('area')['Rate'].agg(['min','max','mean','median']).reset_index()

In [553]:
fileName = 'Area Rate.xlsx'
areaRate.to_excel(os.path.join(saveDir, fileName), index = False)

# Check mothership article
This analysis is inspired by an article on [Low Thia Khiang](https://mothership.sg/2020/05/low-thia-khiang-parliament-attendance/). It states that Mr Low only missed 6 sessions and we do a fact check here. This helps to check the accuracy of our data as well.

Results: Correct. 4 absence starting from 2006. 2 absence in late may which is beyond the date range of this dataset

In [562]:
attendance[attendance.members.str.contains('Low Thia')].groupby('status')['members'].count()

status
Absent       8
Present    412
Name: members, dtype: int64

In [564]:
attendance[(attendance.members.str.contains('Low Thia')) & (attendance.status != 'Present')]

,status,num,fileLink,sittingDate,parliament,area,members,proceedingNum,sittingDt,parliamentNum,minPar
4897,Absent,Votes and Proceedings No. 82,https://www.parliament.gov.sg//docs/default-so...,Sitting on 10 September 2018,13th Parliament,Aljunied,Low Thia Khiang,82,2018-09-10,13,10
7433,Absent,Votes and Proceedings No. 56,https://www.parliament.gov.sg//docs/default-so...,Sitting on 9 January 2018,13th Parliament,Aljunied,Low Thia Khiang,56,2018-01-09,13,10
7530,Absent,Votes and Proceedings No. 55,https://www.parliament.gov.sg//docs/default-so...,Sitting on 8 January 2018,13th Parliament,Aljunied,Low Thia Khiang,55,2018-01-08,13,10
26160,Absent,Votes and Proceedings No. 30,https://www.parliament.gov.sg//docs/default-so...,Sitting on 19 July 2010,11th Parliament,Hougang,Low Thia Khiang,30,2010-07-19,11,10
37391,Absent,Votes and Proceedings No. 28,https://www.parliament.gov.sg//docs/default-so...,Sitting on 19 September 2005,10th Parliament,Hougang,Low Thia Khiang,28,2005-09-19,10,10
37579,Absent,Votes and Proceedings No. 26,https://www.parliament.gov.sg//docs/default-so...,Sitting on 21 July 2005,10th Parliament,Hougang,Low Thia Khiang,26,2005-07-21,10,10
37674,Absent,Votes and Proceedings No. 25,https://www.parliament.gov.sg//docs/default-so...,Sitting on 20 July 2005,10th Parliament,Hougang,Low Thia Khiang,25,2005-07-20,10,10
37772,Absent,Votes and Proceedings No. 24,https://www.parliament.gov.sg//docs/default-so...,Sitting on 19 July 2005,10th Parliament,Hougang,Low Thia Khiang,24,2005-07-19,10,10
